# Plot ISH image from the AIBS website

## Description

The Allen Institute for brain science (AIBS) has performed hundreds of *in situ* hybridization (ISH) experiments on mice (see Lein et al. 2008).

The results of these experiments are high resolution image stacks of coronal or sagittal brain slices, aligned to the AIBS average brain.
These ISH datasets are used in Erö et al. (2018) and Rodarie et al. (2021) to predict the densities of different cell types in the mouse brain.

In this notebook we are plotting different brain slices of GAD1 experiments available at: http://mouse.brain-map.org

In [ ]:
import numpy as np
import json
import urllib
from os.path import join, abspath
import matplotlib.pylab as plt
from matplotlib import colors
from matplotlib.cbook import get_sample_data
from voxcell import VoxelData
from scipy.ndimage import binary_dilation, generate_binary_structure
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

In [ ]:
DATA_FOLDER = 'path/to/data/'
OUTPUT_FOLDER = "path/to/output"

## Set matplotlib parameters

In [ ]:
SMALL_SIZE = 48
MEDIUM_SIZE = 48
BIGGER_SIZE = 48
plt.rcParams["font.family"] = "Arial"
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams['xtick.major.pad']  = 9.0
plt.rcParams['axes.linewidth'] = 2.0
plt.rcParams['xtick.major.size' ] = 7.0*3.0
plt.rcParams['xtick.minor.size' ] = 4.0*3.0
plt.rcParams['ytick.major.size' ] = 7.0*3.0
plt.rcParams['ytick.minor.size' ] = 4.0*3.0
plt.rcParams['xtick.major.width'] = 2.4
plt.rcParams['xtick.minor.width'] = 1.8
plt.rcParams['ytick.major.width'] = 2.4
plt.rcParams['ytick.minor.width'] = 1.8

## Download the ISH images from the AIBS website.

We are downloading the coronal slice 288 of the experiment 79556706: http://mouse.brain-map.org/experiment/show/79556706

In [ ]:
raw_ish_url = "http://mouse.brain-map.org/cgi-bin/imageservice?path=/external/aibssan/production25/stitch1/0301101826/0301101826_1_zoom.aff&mime=1&fileout=79556706_288.jpg&zoom=6&top=0&left=0&width=10209&height=7281"
urllib.request.urlretrieve(raw_ish_url, join(DATA_FOLDER, "raw_GAD1_ISH.jpg"))
expr_ish_url = "http://mouse.brain-map.org/cgi-bin/imageservice?path=/external/aibssan/production32/prod303/0301101826/0301101826_288_expression.aff&mime=1&fileout=79556706_288.jpg&zoom=6&top=0&left=0&width=10209&height=7281&filter=colormap&filterVals=0.5,1,0,256,4"
urllib.request.urlretrieve(expr_ish_url, join(DATA_FOLDER, "expr_GAD1_ISH.jpg"))

## Plot raw ISH slice

In [ ]:
pixel_res = 1.05 # number of um per pixel from AIBS ISH images full res
with get_sample_data(abspath(join(DATA_FOLDER, 'raw_GAD1_ISH.jpg'))) as image_file:
    image = plt.imread(image_file)
image = np.array(image)
image[image>240] = 255 # set background to white

In [ ]:
fig, ax = plt.subplots(figsize=(15,10.5263))
im = ax.imshow(image)
x = 100
y = image.shape[0]*0.85
length = 1000. / pixel_res # convert to mm
ax.hlines(y=y, xmin=x, xmax=x+length, linewidth=15, color='black')
ax.text(x+length//2,y-260, "1 mm", weight='bold', horizontalalignment='center')
ax.axis("off")
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, 'GAD1_slice.png'), dpi=200, facecolor="white")

## Plot filtered ISH slice   

In [ ]:
with get_sample_data(abspath(join(DATA_FOLDER, 'expr_GAD1_ISH.jpg'))) as image_file:
    image = plt.imread(image_file)
image = np.array(image)
image[image<10] = 0
image = 255 - image # invert colors so that background becomes white

In [ ]:
# Recover colormap 
cmap = plt.cm.jet
cmaplist = [cmap(i) for i in range(cmap.N)]
for i_col, _ in enumerate(cmaplist):
    cmaplist[i_col] = (1.0 - cmaplist[i_col][0], 
                       1.0 - cmaplist[i_col][1], 
                       1.0 - cmaplist[i_col][2], 
                       cmaplist[i_col][3])
cmaplist = np.array(cmaplist)[np.linspace(0, len(cmaplist)-1, 10, dtype=int)][1:]
cmap = colors.LinearSegmentedColormap.from_list('mcm',cmaplist, N=len(cmaplist))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10.5263))
im = ax.imshow(image, interpolation="nearest")
x = 100
y = image.shape[0]*0.85
length = 1000. / pixel_res
ax.hlines(y=y, xmin=x, xmax=x+length, linewidth=15, color='black')
ax.text(x+length//2,y-260, "1 mm", weight='bold', horizontalalignment='center')
ax.axis("off")
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, 'GAD_expr_white.png'), dpi=200, facecolor="white")

In [ ]:
# Plot the legend for previous figure in a separate figure.
fig, ax = plt.subplots(figsize=(3, 10.5263))
cbaxes = fig.add_axes([0.2, 0.2, 0.1, 0.6]) 
cbar = plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(0.1, 1.0), cmap=cmap), cax=cbaxes)#, shrink=0.5)
cbar.ax.tick_params(labelsize=30)
cbar.set_label('Expression level', rotation=90)
ax.axis("off")
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, 'colorbar_exp.png'), dpi=200, facecolor="white")

## Plot binarized ISH slices

The filtered images from the AIBS are binarized using a Otsu thresholding. These images will be realigned to the Nissl volume using the Deep-Atlas toolboox: https://github.com/BlueBrain/Deep-Atlas.

In [ ]:
with get_sample_data(abspath(join(DATA_FOLDER, 'expr_GAD1_ISH.jpg'))) as image_file:
    image = plt.imread(image_file)
image = np.array(image)
image[image<10] = 0
image = 255 - image # invert colors so that background becomes white

In [ ]:
expr_preprocessed = rgb2gray(image)
img_binary = (expr_preprocessed > threshold_otsu(expr_preprocessed)) * 1
expr_preprocessed = img_binary.astype("uint8")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10.5263))
ax.imshow(~expr_preprocessed, interpolation="nearest", cmap="Greys")
x = 100
y = image.shape[0]*0.85
length = 1000. / pixel_res
ax.hlines(y=y, xmin=x, xmax=x+length, linewidth=15, color='black')
ax.text(x+length//2,y-260, "1 mm", weight='bold', horizontalalignment='center')
ax.axis("off")
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, 'GAD_binary_white.png'), dpi=200, facecolor="white")

## Plot ISH coronal slice positions

The AIBS provides metadata for their ISH experiments, including the positions of the ISH slices in the mouse brain (according to the AIBS average brain).
These informations can be extracted using the DeepAtlas toolbox: https://github.com/BlueBrain/Deep-Atlas. 

Here we are plotting the slice positions of the coronal ISH experiment (#479) on top of the boundaries of the annotation atlas.

In [ ]:
# List of the ISH slices positions in the annotation atlas after realignment.
GAD_slices = json.loads(open(join(DATA_FOLDER, "realigned_slices.json"), "r").read())["479"]

In [ ]:
# Annotation file
annotation = VoxelData.load_nrrd(join(DATA_FOLDER, "annotations.nrrd")).raw

In [ ]:
struct = generate_binary_structure(2, 2)
thickness = 1 # thickness of the brain outside boundary 
slice_ = int(annotation.shape[2] // 2.5)

lim_GAD=[GAD_slices[0],GAD_slices[-1]]
loc_filt = np.zeros(annotation.shape)
loc_filt[annotation!=0] = 1

loc_filt = loc_filt[:, :, slice_]
boundaries = binary_dilation(loc_filt, struct, iterations=thickness) * (annotation[:, :, slice_]==0)
fig = plt.figure(figsize=(15, 9.1))

plt.imshow(boundaries.T, interpolation="nearest", cmap="Greys")
for i in GAD_slices:
    plt.axvline(x=[i], color=[0, .45, .7], linewidth=1)
y = 280
x = 20
plt.hlines(y=y, xmin=x, xmax=x + 40, linewidth=15, color="black")
plt.text(x + 18, y - 10, "1 mm", weight='bold', horizontalalignment='center')
plt.axis('off')
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, 'GAD_slices_position.png'), dpi=200, facecolor="white")